In [4]:
import fiftyone as fo
from fiftyone import ViewField as F

# Load a small dataset
dataset = fo.zoo.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=10,
)

***Python 3.8 Deprecation Notice***
Python 3.8 will no longer be supported in new releases after October 1, 2024.
Please upgrade to Python 3.9 or later.
For additional details please see https://deprecation.voxel51.com


Found annotations at '/home/lradovan/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation-10'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
from libcom import FOPAHeatMapModel
from libcom.utils.process_image import make_image_grid, draw_bbox_on_image
import cv2
import os

fg_img = '/home/lradovan/workspace/libcom/tests/source/foreground/1.jpg'
fg_mask = '/home/lradovan/workspace/libcom/tests/source/foreground_mask/1.png'

#test_set = get_test_list_fopa_heatmap()

test_set = []
for sample in dataset[5:6]:
    print(sample.metadata.width)
    test_set.append({'foreground': fg_img, 'foreground_mask': fg_mask, 'background': sample.filepath})
    
result_dir = './results/'
net = FOPAHeatMapModel(device=0)

for pair in test_set[:1]:
    fg_img, fg_mask, bg_img = pair['foreground'], pair['foreground_mask'], pair['background']
    bboxes, heatmaps = net(fg_img, fg_mask, bg_img, cache_dir=os.path.join(result_dir, 'cache'), heatmap_dir=os.path.join(result_dir, 'heatmap'))
    img_name  = os.path.basename(bg_img).replace('.png', '.jpg')
    bbox = bboxes[0]
    bg_img  = draw_bbox_on_image(bg_img, [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
    grid_img  = make_image_grid([bg_img, fg_img, heatmaps[0]])
    os.makedirs(os.path.join(result_dir, 'grid'), exist_ok=True)
    res_path  = os.path.join(result_dir, 'grid', img_name)
    cv2.imwrite(res_path, grid_img)
    print('save result to ', res_path)
print(f'end testing!')

640
save result to  ./results/grid/000000000785.jpg
end testing!


In [6]:
# we need some way to get the bounding boxes for the occluders....

def get_bbox_list(occludee_bbs, bg_img, fg_w, fg_h):
    # for occludee_bb in occludee_bbs:
    return 0

In [ ]:
# IN PROGRESS

from libcom import OPAScoreModel
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid
import cv2

net = OPAScoreModel(device=0, model_type='SimOPA')

# generate multiple different foreground scales

# for each foreground scale, try different locations that allow the bounding box to overlap with the background targets bb

# for each scale, take the highest OPA score

# take the scaled image with the highest OPA score

from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales

scaled_fg_dir, scaled_mask_dir, csv_file = prepare_multi_fg_scales(cache_dir, foreground_image, foreground_mask, background_image, fg_scale_num)
 
# iterate over the different foreground scales
with open(csv_path, mode='r', newline='') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    score = 0
    for row in csv_reader:
        fg_name   = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
        mask_name = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
        scale     = row['scale']
        fg_w = int(row['newWidth'])
        fg_h = int(row['newHeight'])
        save_name = fg_name.split(".")[0] + '_' + str(scale) + '.jpg'

        bg_img    = read_image_pil(background_image)
        fg_img    = read_image_pil(os.path.join(scaled_fg_dir, fg_name))
        mask      = read_mask_pil(os.path.join(scaled_mask_dir, mask_name))

        bbox_list = get_bbox_list(occluded_bb, bg_img, fg_w, fg_h)

        for bbox in bbox_list:
            comp, comp_mask = get_composite_image(fg_img, fg_mask, bg_img, bbox_list[0])
            bbox_score = net(comp, comp_mask)
            if bbox_score > score:
                res = comp, comp_mask
                score = bbox_score

        grid_img  = make_image_grid(res)
        cv2.imwrite('/home/lradovan/results/', grid_img)